# ***Adaptive Retrieval-Augmented Generation (RAG)***

Simplified idea from the paper https://arxiv.org/abs/2403.14403 and https://arxiv.org/abs/2402.16457.
**Adaptive Retrieval-Augmented Generation (Adaptive RAG)** is an advanced approach to information retrieval and response generation that dynamically adjusts its strategy based on the nature of the query. This method enhances the traditional RAG system by tailoring the retrieval process to different types of questions, resulting in more accurate and contextually relevant responses.

**Key Features of Adaptive RAG:**
- Query Classification: Automatically categorizes queries into types such as factual, analytical, opinion-based, or contextual.
- Dynamic Retrieval: Adjusts the number and type of documents retrieved based on the query classification and the model's confidence in answering.
- Efficient Resource Use: Optimizes computational resources by retrieving only as much information as necessary to answer the query confidently.
- Improved Accuracy: By adapting to each query type, the system can provide more precise and relevant information.

**How It Works:**
- The system first classifies the incoming query.
- Based on the classification, it selects an appropriate retrieval strategy.
- The retrieval process (score threshold = 0.9) starts with a minimal set of documents and expands as needed.
- If no documents are returned from the vector search , a websearch is launched to ensure the question has an answer.
- The retrieved documents are reranked in the order suitable to relevance to the query
- The Language Model (LLM) attempts to answer the query with the retrieved information.

Adaptive RAG represents a significant advancement in RAG systems, offering a more flexible and efficient approach to handling diverse query types while maintaining high accuracy.

This notebook uses [Qdrant](https://qdrant.tech/) with [Fastembed](https://qdrant.github.io/fastembed/) as a vector store , [Synthetic dataset](https://huggingface.co/datasets/atitaarora/adaptive_dataset) as source data, [OpenAI](https://openai.com/) as LLM and [Tavily](https://docs.tavily.com/docs/python-sdk/tavily-search/api-reference) for websearch capabilities.

### **1. Installs**

In [135]:
#!pip install qdrant
#!pip install openai
#!pip install datasets
#!pip install "qdrant-client[fastembed]"
#!pip install tavily-python

### **2. Import Dependencies**

In [137]:
import os
import sys
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import PointStruct
from openai import OpenAI
import numpy as np
from tavily import TavilyClient

### **3. Settings and Configurations**

In [158]:
# Set the OpenAI API key environment variable
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")

#Set up Tavily API for websearch
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

#Collection Name for the notebook
COLLECTION_NAME = "adaptive_retrieval"

# Initialize Qdrant client
client = QdrantClient(location=":memory:")

# Initialize OpenAI client
openai_client = OpenAI()

#Initialise Tavily client
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

In [162]:
## Check if Collection already exists
client.get_collections()

CollectionsResponse(collections=[])

### **4. Dataset config to be used in the notebook**

In [159]:
from datasets import load_dataset
dataset = load_dataset("atitaarora/adaptive_dataset", split="train")

In [160]:
len(dataset)

8

In [5]:
dataset[2]

{'content': 'title: Climate Change Impact Analysis,content: Climate change is causing significant environmental shifts globally. Rising temperatures lead to melting ice caps, rising sea levels, and more frequent extreme weather events. This affects ecosystems, agriculture, and human settlements. Mitigation strategies include reducing greenhouse gas emissions and developing sustainable energy sources.',
 'metadata': {'content': 'Climate change is causing significant environmental shifts globally. Rising temperatures lead to melting ice caps, rising sea levels, and more frequent extreme weather events. This affects ecosystems, agriculture, and human settlements. Mitigation strategies include reducing greenhouse gas emissions and developing sustainable energy sources.',
  'context': None,
  'title': 'Climate Change Impact Analysis'}}

### **5. Process dataset for ingestion**

In [163]:
from tqdm import tqdm
from langchain.docstore.document import Document as LangchainDocument

## Dataset to langchain document
langchain_docs = [
    LangchainDocument(page_content=doc["content"], metadata=doc["metadata"])
    for doc in tqdm(dataset)
]

len(langchain_docs)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 620.45it/s]


8

In [7]:
langchain_docs[2]

Document(metadata={'content': 'Climate change is causing significant environmental shifts globally. Rising temperatures lead to melting ice caps, rising sea levels, and more frequent extreme weather events. This affects ecosystems, agriculture, and human settlements. Mitigation strategies include reducing greenhouse gas emissions and developing sustainable energy sources.', 'context': None, 'title': 'Climate Change Impact Analysis'}, page_content='title: Climate Change Impact Analysis,content: Climate change is causing significant environmental shifts globally. Rising temperatures lead to melting ice caps, rising sea levels, and more frequent extreme weather events. This affects ecosystems, agriculture, and human settlements. Mitigation strategies include reducing greenhouse gas emissions and developing sustainable energy sources.')

### **6. Document Embedding configuration and processing**

In [164]:
from fastembed.embedding import TextEmbedding
embedding_model = TextEmbedding()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

In [165]:
def process_document_embeddings(docs):
    docs_contents = []
    docs_metadatas = []

    for doc in docs:
        if hasattr(doc, 'page_content') and hasattr(doc, 'metadata'):
            docs_contents.append(doc.page_content)
            docs_metadatas.append(doc.metadata)
        else:
            # Handle the case where attributes are missing
            print("Warning: Some documents do not have 'content' or 'metadata' attributes.")
    
    print("data-size : ",len(docs))
    print("content : ",len(docs_contents))
    print("metadata : ",len(docs_metadatas))
    return (docs_contents,docs_metadatas)

In [166]:
docs_contents , docs_metadatas = process_document_embeddings(langchain_docs)

data-size :  8
content :  8
metadata :  8


### **7. Adding documents to Qdrant**

In [167]:
client.add(collection_name=COLLECTION_NAME, metadata=docs_metadatas, documents=docs_contents)

['a7655fe59a7749668e723a110f53193b',
 '232b75f8f68d49caac14cfd37e62a33d',
 '130e2292a9ff4019bf55b97979e6fca5',
 '5cc839972ff64466879057b0d218e8d3',
 'fec038990b0e40b98979574de447bd5e',
 '5421d47447aa42f9b1c55853c097902d',
 '0b3e0c9c8e624dc087fa0297c7f8836e',
 '315c1d6d875a47e18329ee52fb3d2d81']

In [168]:
## Test query
search_result = client.query(
    collection_name=COLLECTION_NAME,
    query_text="solar system",
    limit=2
)
print(search_result)

[QueryResponse(id='a7655fe59a7749668e723a110f53193b', embedding=None, sparse_embedding=None, metadata={'document': 'title: The Solar System, content: The solar system consists of the Sun and everything that orbits around it, including eight planets, dwarf planets, and countless smaller objects. The planets in order from the Sun are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.', 'content': 'The solar system consists of the Sun and everything that orbits around it, including eight planets, dwarf planets, and countless smaller objects. The planets in order from the Sun are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.', 'context': None, 'title': 'The Solar System'}, document='title: The Solar System, content: The solar system consists of the Sun and everything that orbits around it, including eight planets, dwarf planets, and countless smaller objects. The planets in order from the Sun are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and

### **8. Building Adaptive RAG blocks**

### **classify_query()**

Method to classify the query as Factual , Analytical , Opinion bases or Contextual to user history so it can be processed accordingly.

In [36]:
def classify_query(query):
    prompt = f"Classify the following query into one of these categories: Factual, Analytical, Opinion, or Contextual.\nQuery: {query}\nCategory:"
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    print("Identified query type: " , response.choices[0].message.content.strip())
    return response.choices[0].message.content.strip()

### **web_search()**

Method to launch the websearch for the given query to ensure it is answered with latest information

In [183]:
def web_search(query):
    #context = tavily_client.get_search_context(query,max_results=2,results=["title","content","score"])
    context = tavily_client.search(query,search_depth="advanced", max_results=2,results=["title","content","score"])["results"]
    #print(context)
    return context

In [181]:
#web_search("What happened during the Burning Man floods?")

### **factual_strategy()**

Method to improvise/enhance the user query using LLM and launch a factual/factoid search that can be supported through direct vector search from our vector store.


In [173]:
def factual_strategy(query):
    enhanced_query = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": f"Enhance this query for better precision: {query}"}]
    ).choices[0].message.content
    
    search_result = client.query(
        collection_name=COLLECTION_NAME,
        query_text=query,
        limit=2,
        score_threshold=0.9
    )
    
    return [hit.metadata for hit in search_result]

### **analytical_strategy()**

Method to analyse the user query using LLM where it generates 2 sub queries to cover different aspects of the questions followed by semantic search to our vector store.

In [111]:
def analytical_strategy(query):
    sub_queries = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": f"Generate 2 sub-queries to cover different aspects of this query: {query}"}]
    ).choices[0].message.content.split("\n")
    print(sub_queries)
    all_results = []
    for sub_query in sub_queries:
        search_result = client.query(
            collection_name=COLLECTION_NAME,
            query_text=sub_query,
            limit=1
        )
        all_results.extend([hit.metadata for hit in search_result])
    
    return all_results

### **opinion_strategy()**

Method to generate 2 different viewpoints of the user query using LLM and collect respective results for each viewpoint through semantic search from our vector store to prepare a rounded answer to the user query.

In [126]:
def opinion_strategy(query):
    viewpoints = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": f"Identify 2 different viewpoints on this topic: {query}"}]
    ).choices[0].message.content.split("\n")
    
    all_results = []
    for viewpoint in viewpoints:
        search_result = client.query(
            collection_name=COLLECTION_NAME,
            query_text=viewpoint,
            limit=1
        )
        all_results.extend([hit.metadata for hit in search_result])
    
    return all_results

### **contexual_strategy()**

Method to accomodate contexual information along with the user query to filter and process the most appropriate responses to the user query using semantic search with filters from our vector store.

In [127]:
from qdrant_client.models import Filter, FieldCondition, Range

def contextual_strategy(query, user_context):
    contextualized_query = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": f"Incorporate this user context into the query: {user_context}\nQuery: {query}"}]
    ).choices[0].message.content
    
    if 'location' in context:
        city_filter = models.Filter(
            must=[
                models.FieldCondition(
                    key="context.location", 
                    match=models.MatchValue(value=context['location'])
                )
            ]
        )
    
    search_result = client.query(
        collection_name=COLLECTION_NAME,
        query_text=query,
        query_filter=city_filter,
        limit=2
    )
    print(search_result)
    return [hit.metadata for hit in search_result]


### **rank_documents()**

Rerank the documents retrieved from the vector store ,  per the perceived relevance from LLM leveraging user query.

Note : Can leverage reranker model from providers like Jina , Cohere and Mixedbread.

### **generate_response()** 

Bring together the relevent retrieved context from vector store / web to generate a suitable answer for user query.

Note : Can leverage Prompt Experimentation, if needed.

In [108]:
def rank_documents(documents, query):
    document_contents = [doc['content'] for doc in documents]
    ranked_docs = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"Rank these documents by relevance to the query: {query}\n\nDocuments:\n" + "\n".join(document_contents)}
        ]
    ).choices[0].message.content.split("\n")
    return ranked_docs

def generate_response(query, context):
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Use the provided context to answer the user's query."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuery: {query}"}
        ]
    )
    return response.choices[0].message.content

### **9. Adaptive RAG Pipeline**

This brings together all our bit and pieces together to form a desired RAG pipeline that adapts per the user query and answers them in the most appropriate way.

In [175]:
def adaptive_rag(query, user_context=None):
    query_type = classify_query(query)
    
    if query_type == "Factual":
        retrieved_docs = factual_strategy(query)
    elif query_type == "Analytical":
        retrieved_docs = analytical_strategy(query)
    elif query_type == "Opinion":
        retrieved_docs = opinion_strategy(query)
    elif query_type == "Contextual":
        retrieved_docs = contextual_strategy(query, user_context)
    else:
        raise ValueError("Invalid query type")
    
    if len(retrieved_docs) == 0:
        retrieved_docs = web_search(query)
        
    ranked_docs = rank_documents(retrieved_docs, query)
    context = "\n".join(ranked_docs[:2])  # Use top 2 ranked documents
    print("context: " , context)
    response = generate_response(query, context)
    return response

### **10. Experiments to test our Adapative RAG Pipeline**

In [184]:
## The one with Websearch
query_test = "What happened on 22nd oct 2024"
classify_query(query_test)
response_test = adaptive_rag(query_test)
print(response_test)


Identified query type:  Factual
Identified query type:  Factual
context:  1. Oct. 22, 2024 / 3:00 AM UPI Almanac for Tuesday, Oct. 22, 2024 - Mentions the date in question
2. Updated October 22, 2024 at 11:30 a.m.U.S. President John F. Kennedy announced that American reconnaissance planes discovered Soviet nuclear weapons in Cuba, marking the beginning of the Cuban Missile Crisis on this day in 1962. - Mentions a significant event that happened on the same date in history
On October 22, 2024, the UPI Almanac mentions the date itself. Additionally, that day in history, on the same date in 1962, U.S. President John F. Kennedy announced the beginning of the Cuban Missile Crisis after American reconnaissance planes discovered Soviet nuclear weapons in Cuba.


In [37]:
## Example usage with Analytical Strategy
query = "What are the pros and cons of renewable energy?"
response = adaptive_rag(query)
print(response)

Identified query type:  Analytical
context:  1. Climate Change Impact Analysis
2. The Future of Work: Remote vs. Office-based
Pros of renewable energy:
1. Environmentally friendly: It helps reduce greenhouse gas emissions that contribute to climate change.
2. Renewable and sustainable: Resources like sunlight, wind, and water are continuously replenished.
3. Energy security: It reduces dependence on finite fossil fuel resources.
4. Job creation: The renewable energy sector creates jobs and promotes economic growth.
5. Cost-effective: Over time, renewable energy can be more cost-effective than fossil fuels.

Cons of renewable energy:
1. Intermittency: Some renewable sources like wind and solar energy can be intermittent, depending on weather conditions.
2. Initial costs: The upfront costs of installing renewable energy systems can be expensive.
3. Land use: Some renewable energy projects require significant land use, which may impact ecosystems.
4. Energy storage: Storage technology nee

In [109]:
## Example usage with Contexual Strategy
query_2 = "What are good restaurants per my context?"
user_context = {"location":"New York"}
response_2 = adaptive_rag(query_2,user_context)
print(response_2)

Identified query type:  Contextual
[QueryResponse(id='4080153504ae4b5f921c64dcb487644f', embedding=None, sparse_embedding=None, metadata={'document': "title: Vegetarian Restaurants in London,content: London boasts numerous vegetarian dining options. Mildreds offers diverse international cuisine. The Gate serves gourmet vegetarian dishes. For quick bites, try Pret A Manger's vegetarian options. Many traditional pubs now offer extensive vegetarian menus to cater to changing dietary preferences.", 'content': "London boasts numerous vegetarian dining options. Mildreds offers diverse international cuisine. The Gate serves gourmet vegetarian dishes. For quick bites, try Pret A Manger's vegetarian options. Many traditional pubs now offer extensive vegetarian menus to cater to changing dietary preferences.", 'context': {'dietary_preference': 'vegetarian', 'location': 'London'}, 'title': 'Vegetarian Restaurants in London'}, document="title: Vegetarian Restaurants in London,content: London boast

In [132]:
## Example usage with Opinion Based Strategy
query_6 = "Is remote work better or office based?"
response_6 = adaptive_rag(query_6)
print(response_6)

Identified query type:  Opinion
context:  1. The COVID-19 pandemic has accelerated the shift towards remote work. Proponents argue it offers better work-life balance and increased productivity. Critics contend it can lead to isolation and hinder collaboration. The future likely involves a hybrid model, balancing the benefits of both remote and office-based work.
The debate of whether remote work is better than office-based work depends on different perspectives and individual preferences. Remote work can offer benefits such as better work-life balance, increased productivity, and decreased commuting time. On the other hand, office-based work can foster better collaboration and social interaction among colleagues. The future of work is likely to involve a hybrid model that combines the advantages of both remote and office-based work. Ultimately, the best approach may vary depending on the nature of the job, individual preferences, and organizational dynamics.


In [116]:
## Example usage with Factual Strategy
query_4 = "Tell me a bit about World War II Timeline?"
response_4 = adaptive_rag(query_4)
print(response_4)

Identified query type:  Factual
context:  1. World War II lasted from 1939 to 1945. Key events include Germany's invasion of Poland in 1939, the attack on Pearl Harbor in 1941, D-Day in 1944, and the atomic bombings of Hiroshima and Nagasaki in 1945. 

World War II lasted from 1939 to 1945 and consisted of several key events that shaped the conflict. Some of the major events in the World War II timeline include Germany's invasion of Poland in 1939, the attack on Pearl Harbor by Japan in 1941, D-Day in 1944, which marked the Allied invasion of Normandy, France, and the atomic bombings of Hiroshima and Nagasaki in 1945 by the United States. These events are crucial to understanding the progression and impact of World War II.


In [112]:
## Another Example usage with Analytical Strategy
query_3 = "How is automation affecting our work?"
response_3 = adaptive_rag(query_3)
print(response_3)

Identified query type:  Analytical
['Sub-query 1: What are the specific tasks being automated in different industries?', '', 'SELECT industry, automation_impact', 'FROM automation_data', "WHERE automation_impact = 'positive';", '', 'Sub-query 2: How are workers adapting to automation in the workplace?', '', 'SELECT worker_response', 'FROM automation_impact', "WHERE automation_impact = 'negative';"]
context:  1. Automation is reshaping the job market. While it increases productivity and efficiency, it also displaces certain types of jobs. This leads to a shift in skill requirements, with a growing demand for tech-savvy workers. The impact varies across industries, with manufacturing and retail being significantly affected.

Automation is significantly impacting the job market by increasing productivity and efficiency. However, it also results in the displacement of certain jobs, leading to a shift in skill requirements. This shift is causing a growing demand for tech-savvy workers. The 

In [117]:
## Another Example usage with Analytical Strategy
query_5 = "Is rising temparature impacting climate change?"
response_5 = adaptive_rag(query_5)
print(response_5)

Identified query type:  Analytical
['Sub-query 1: ', '- Evaluate the correlation between rising temperatures and greenhouse gas emissions over time. This sub-query would involve analyzing historical data on global temperatures and greenhouse gas emissions to determine if there is a significant relationship between the two factors.', '', 'Sub-query 2: ', '- Examine the impact of rising temperatures on key environmental indicators such as melting ice caps, rising sea levels, and shifts in animal habitats. This sub-query would involve studying environmental data and research studies to assess the direct effects of rising temperatures on various aspects of the climate and ecosystem.']
context:  1. Climate change is causing significant environmental shifts globally. Rising temperatures lead to melting ice caps, rising sea levels, and more frequent extreme weather events. This affects ecosystems, agriculture, and human settlements. Mitigation strategies include reducing greenhouse gas emissi